# Collect data from lab tests

## NOTE: This file depends on the output of the FLOW1 notebook

In [80]:
import pandas as pd
import pickle
import numpy as np
import itertools
import datetime

# Database libraries
import psycopg2

# Stats libraries
from tableone import TableOne
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
# Image libraries
# https://jakevdp.github.io/pdvega/
# jupyter nbextension enable vega3 --py --sys-prefix
import matplotlib.pyplot as plt
import pdvega 
%matplotlib inline

In [46]:
# Create a database connection
# Replace user and password with credentials
user = 'xxx'
password = 'xxx'
host = 'hst953.csail.mit.edu'
dbname = 'mimic'
schema = 'mimiciii'

In [70]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password=password)
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

## Load the previously extracted data_set

In [3]:
data_sl = pickle.load(open("d_slices_5060_g.p","rb"))

In [4]:
data_mro = pickle.load(open("d_mro_5060_g.p","rb"))

In [18]:
#Transform the list of series into a single series
data_sl = pd.concat(data_sl)

In [19]:
data_mro

,subject_id,hadm_id,icustay_id,gender,admittime_hospital,dischtime_hospital,los_hospital,age,admission_type,hospital_expire_flag,...,los_icu,hospstay_seq,icustay_seq,first_careunit,last_careunit,first_wardid,last_wardid,icustay_expire_flag,oasis,oasis_prob
0,285,165312,238023,M,2152-09-21 22:47:00,2152-10-20 15:19:00,28.6889,45.3561,EMERGENCY,0,...,28.6880,1,1,TSICU,TSICU,14,14,0,37,0.188911
1,293,125963,263651,F,2125-02-09 07:15:00,2125-02-13 13:45:00,4.2708,46.9889,ELECTIVE,0,...,1.0570,1,1,CSRU,CSRU,12,12,0,34,0.137099
2,310,142159,269885,M,2139-05-06 13:15:00,2139-05-10 12:00:00,3.9479,42.8033,ELECTIVE,0,...,1.6464,1,1,SICU,SICU,57,57,0,12,0.009522
3,319,124954,281554,F,2156-08-12 14:01:00,2156-08-16 17:52:00,4.1604,47.2634,EMERGENCY,0,...,2.3471,1,1,MICU,MICU,50,50,0,28,0.068843
4,326,188926,281585,M,2118-11-01 21:46:00,2118-11-05 14:19:00,3.6896,41.9747,EMERGENCY,0,...,3.6877,1,1,MICU,MICU,15,15,0,34,0.137099
5,327,146750,272596,F,2194-04-28 09:15:00,2194-04-29 15:00:00,1.2396,42.0937,ELECTIVE,0,...,0.8932,1,1,MICU,MICU,52,52,0,19,0.022930
6,78,100536,233150,M,2177-02-14 00:16:00,2177-02-17 22:12:00,3.9139,48.6253,EMERGENCY,0,...,1.4891,1,1,MICU,MICU,12,12,0,31,0.097783
7,80,115385,256068,M,2148-11-11 12:12:00,2148-11-16 11:53:00,4.9868,43.9147,EMERGENCY,0,...,1.1901,1,1,TSICU,TSICU,33,33,0,23,0.037612
8,86,190243,206222,F,2146-04-06 08:00:00,2146-04-10 14:28:00,4.2694,45.2641,ELECTIVE,0,...,0.9411,1,1,CSRU,CSRU,12,12,0,21,0.029395
9,95,160891,216431,M,2157-12-25 16:28:00,2157-12-27 15:25:00,1.9563,44.1616,EMERGENCY,0,...,0.7381,1,1,CCU,CCU,57,57,0,19,0.022930


In [36]:
RN_np = []
RN_nm = []
OTH_np = []
OTH_nm = [] 
MDs_np = []
MDs_nm = []
RO_np = []
RO_nm = []
total_l = []
for s in data_sl:
    RN_np.append(s[0]['RN'][0])  
    OTH_np.append(s[0]['OTH'][0])
    MDs_np.append(s[0]['MDs'][0]) 
    RO_np.append(s[0]['RO'][0])
    RN_nm.append(s[0]['RN'][1])  
    OTH_nm.append(s[0]['OTH'][1])
    MDs_nm.append(s[0]['MDs'][1]) 
    RO_nm.append(s[0]['RO'][1])
    total_l.append(s[1])

In [37]:
data_mro_plus = data_mro

In [41]:
data_mro_plus["RN_np"] = RN_np
data_mro_plus["RN_nm"] = RN_nm
data_mro_plus["RO_np"] = RO_np
data_mro_plus["RO_nm"] = RO_nm
data_mro_plus["MDs_nm"] = MDs_nm
data_mro_plus["MDs_np"] = MDs_np
data_mro_plus["OTH_nm"] = OTH_nm
data_mro_plus["OTH_np"] = OTH_np
data_mro_plus["MeasTot"] = total_l

In [42]:
data_mro_plus

,subject_id,hadm_id,icustay_id,gender,admittime_hospital,dischtime_hospital,los_hospital,age,admission_type,hospital_expire_flag,...,oasis_prob,RN_np,RN_nm,RO_np,RO_nm,MDs_nm,MDs_np,OTH_nm,OTH_np,MeasTot
0,285,165312,238023,M,2152-09-21 22:47:00,2152-10-20 15:19:00,28.6889,45.3561,EMERGENCY,0,...,0.188911,30,34835,0,0,0,0,11014,40,45849
1,293,125963,263651,F,2125-02-09 07:15:00,2125-02-13 13:45:00,4.2708,46.9889,ELECTIVE,0,...,0.137099,5,1086,0,0,0,0,487,5,1573
2,310,142159,269885,M,2139-05-06 13:15:00,2139-05-10 12:00:00,3.9479,42.8033,ELECTIVE,0,...,0.009522,6,2432,0,0,0,0,83,3,2515
3,319,124954,281554,F,2156-08-12 14:01:00,2156-08-16 17:52:00,4.1604,47.2634,EMERGENCY,0,...,0.068843,6,2497,0,0,0,0,81,4,2578
4,326,188926,281585,M,2118-11-01 21:46:00,2118-11-05 14:19:00,3.6896,41.9747,EMERGENCY,0,...,0.137099,9,4152,0,0,0,0,503,8,4655
5,327,146750,272596,F,2194-04-28 09:15:00,2194-04-29 15:00:00,1.2396,42.0937,ELECTIVE,0,...,0.022930,3,603,0,0,0,0,39,2,642
6,78,100536,233150,M,2177-02-14 00:16:00,2177-02-17 22:12:00,3.9139,48.6253,EMERGENCY,0,...,0.097783,2,323,0,0,0,0,293,2,616
7,80,115385,256068,M,2148-11-11 12:12:00,2148-11-16 11:53:00,4.9868,43.9147,EMERGENCY,0,...,0.037612,3,827,0,0,0,0,943,2,1770
8,86,190243,206222,F,2146-04-06 08:00:00,2146-04-10 14:28:00,4.2694,45.2641,ELECTIVE,0,...,0.029395,3,703,0,0,0,0,801,5,1504
9,95,160891,216431,M,2157-12-25 16:28:00,2157-12-27 15:25:00,1.9563,44.1616,EMERGENCY,0,...,0.022930,4,504,0,0,0,0,45,2,549


In [59]:
in_t

Timestamp('2125-02-09 10:35:19')

In [83]:
data_mro_plus.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            5618, 5620, 5621, 5624, 5625, 5626, 5627, 5629, 5630, 5631],
           dtype='int64', length=4224)

In [87]:
tot_tests = []

for i,row in data_mro_plus.iterrows():
    s=datetime.datetime.now()
    sub_id = int(row['subject_id'])
    out_t = row['outtime_icu']
    in_t = row['intime_icu']
    query = \
    """
        SELECT l.subject_id, l.charttime
        FROM labevents l
        WHERE l.subject_id = %(subj)s AND l.charttime < %(endt)s AND l.charttime > %(startt)s
    """
    
    data_lbtest = pd.read_sql_query(query,con,params={'subj' : sub_id , 'endt' : out_t , 'startt': in_t })
    tot_tests.append(len(data_lbtest))
    e=datetime.datetime.now()
    print(i,(e-s).total_seconds())

0 0.253005
1 0.084569
2 0.078286
3 0.083747
4 0.084256
5 0.068684
6 0.084675
7 0.078136
8 0.099443
9 0.084389
10 0.084392
11 0.169258
12 0.084123
13 0.078508
14 0.084364
15 0.084485
16 0.08431
17 0.08436
18 0.099957
19 0.084123
20 0.115649
21 0.100088
23 0.600028
24 0.099902
25 0.331202
26 0.315334
27 0.184306
29 0.552943
30 0.677905
31 0.837377
34 1.000114
35 0.284101
36 0.262523
37 0.600576
38 0.168708
39 0.900116
40 1.614521
41 0.168726
42 0.100134
43 0.584043
44 0.363027
50 1.25287
51 0.215601
52 0.11553
53 0.084369
2950 0.368537
56 0.298368
57 0.117292
58 0.084058
59 0.162699
60 0.237105
61 0.099913
62 0.500329
64 0.078272
65 0.152515
66 0.115714
67 0.284259
68 0.146831
70 2.418534
71 0.165354
72 0.384341
74 0.930998
2976 0.584509
77 0.084184
78 0.368854
79 0.084125
80 0.131476
81 0.364283
82 0.920035
83 0.884044
84 0.431179
85 0.984267
86 0.11555
87 0.931197
88 0.184382
89 0.152922
90 0.178427
92 0.668525
93 0.21557
94 0.084254
95 0.469052
96 0.115573
97 0.199803
98 0.099975
99 0

794 0.100283
795 19.51794
796 7.736389
797 27.028702
3673 0.53071
799 0.537357
3649 2.362044
801 0.153248
802 0.415593
803 0.099835
804 0.868733
805 0.115535
806 0.515716
807 0.152803
808 0.24715
809 0.231197
810 0.320108
811 0.217335
812 0.246838
813 0.099988
814 0.835627
815 0.348157
816 0.16872
817 1.778763
818 0.167327
819 0.353131
820 0.262711
823 0.20004
824 0.237372
825 0.246737
826 0.337133
827 0.17822
828 0.099968
829 0.100004
830 0.337948
832 0.082055
833 0.301775
834 0.1312
835 0.215562
836 0.315543
837 0.600022
839 0.699875
840 0.116002
841 0.099962
842 0.299649
843 0.137416
844 0.762389
845 1.099701
846 0.268801
847 0.732588
848 0.153017
849 0.151322
850 0.264515
851 1.336683
852 0.34672
853 0.15141
854 0.563624
855 0.400154
856 0.615366
857 0.119184
858 0.217774
859 0.215551
860 0.415458
861 0.268779
862 0.379524
863 0.436012
865 0.346704
3716 45.295864
868 0.837119
869 0.261969
870 0.484388
871 1.384886
876 1.015515
877 1.231835
881 3.0384
882 0.562456
883 0.168588
884 0

1526 0.484365
1527 0.231202
1528 0.183927
1529 0.284805
1530 1.268424
1531 0.246834
1532 0.753008
1533 0.415984
1535 0.400038
1536 0.246821
1537 0.284257
1538 0.184198
1539 0.315985
1543 0.283757
1544 0.147383
1545 0.199914
1546 1.568958
1547 0.385079
1548 0.366282
1549 0.534068
1550 0.16713
1551 0.679379
1552 1.237644
1553 0.547219
1554 0.515716
1555 0.269043
1556 0.131194
1557 0.299948
1558 0.236942
1559 0.56368
1560 0.121825
1561 0.394584
1563 0.235799
1564 3.701486
1565 0.346973
1566 0.953368
1567 0.284194
1568 0.447109
1569 1.137251
1571 0.23139
1572 0.315593
1574 1.568495
1575 1.199787
1577 0.784158
1579 1.362572
4412 1.636052
1584 0.317161
1585 0.699532
1586 0.099775
1587 0.331334
1589 0.221882
1590 0.215978
1591 0.262279
1592 0.237528
1593 0.399987
1595 0.262202
1596 1.62177
1597 0.499957
1598 0.247039
1599 0.384209
1602 1.369323
1604 0.078263
1605 1.736673
1606 1.03093
1607 0.552271
1608 1.115854
1610 0.146865
1611 0.153114
1612 0.515571
1615 0.431038
1616 0.999988
1618 2.4153

2233 0.467577
2234 0.115942
2235 0.468511
2236 0.268702
2237 0.146858
2238 0.35319
2239 0.146806
2240 0.237263
2241 0.178151
2242 0.199993
2243 0.321677
2244 0.147097
2245 0.268606
2246 0.199881
2247 0.216038
2248 0.099968
2249 0.084306
2250 0.515429
2251 0.431189
2252 0.815425
2253 0.084471
2254 0.584192
2255 0.384347
2256 0.184467
2257 0.347036
2258 0.115464
2259 0.120589
2260 0.2168
2261 0.084212
2262 0.378102
2263 0.121741
2264 0.277839
2265 0.084538
2267 0.153159
2268 0.084282
2269 0.100068
2270 0.231382
2271 0.400088
2272 0.765885
2273 0.218373
2274 0.131231
2275 0.184362
2276 0.146746
2277 0.284425
2278 0.13423
2279 0.150011
2280 0.084445
2282 0.200031
2285 0.199973
2286 0.531167
2288 0.484329
2289 1.084093
2291 0.115778
2292 0.283978
2293 1.016164
2294 0.115555
2295 0.652776
2296 0.146854
2297 0.253077
2298 0.246869
2299 0.652896
2300 0.800059
2301 0.147186
2302 0.135129
2303 0.27969
2304 0.137535
2305 0.131164
2306 0.715528
2307 0.215489
2309 0.421638
2310 0.394398
2311 0.4056

2922 0.084128
2924 0.27813
2925 0.184442
2926 0.137339
2927 0.594314
2928 0.136969
2929 0.101504
2932 0.267621
2933 0.099999
2934 0.078226
2935 0.468269
2936 0.099845
2937 0.068733
2941 0.084872
2942 0.461862
2943 0.221805
2944 0.178194
2945 0.099922
2946 0.100045
2947 0.221869
2949 0.099913
2953 0.53143
2954 1.99975
2955 0.446704
2957 0.153224
2958 0.099912
2959 0.084433
2960 0.200038
2963 0.499881
2964 0.146852
2966 0.099721
2968 0.615565
2969 0.153053
2970 0.231376
2971 0.13799
2972 0.096812
2973 0.202528
2974 1.031015
2975 0.084297
2979 0.162498
2980 0.25336
2981 0.115727
2982 0.115553
2983 0.19976
2985 0.468997
2987 0.72284
2988 0.092826
2989 0.500853
2991 0.514908
2992 0.116994
2993 0.299666
2994 0.201863
2995 0.264884
2996 2.252589
2997 0.631367
2998 0.184361
2999 0.100005
3000 0.21546
3001 0.084228
3003 0.14685
3005 0.337322
3006 0.499968
3007 3.061533
3008 1.484367
3009 0.199827
3011 0.115579
3012 0.200434
3014 0.137162
3016 0.146853
3017 0.099999
3018 0.315345
3019 0.084842
3

3835 0.431047
3836 0.099989
3837 0.084365
3838 0.136258
3839 0.478585
3840 0.421938
3841 0.69991
3845 0.078211
3851 0.199988
3852 0.50026
3857 0.237285
3858 0.146711
3859 0.068887
3860 0.584304
3861 0.199973
3862 1.283984
3864 0.199982
3865 0.231794
3866 0.215108
3867 0.115749
3868 0.084386
3869 0.815514
3871 0.899896
3872 0.08438
3873 0.284336
3874 0.284329
3875 0.962261
3876 0.33754
3879 0.100037
3880 0.084126
3881 0.262705
3883 0.237379
3884 0.147187
3885 0.315584
3887 0.215579
3889 0.099993
3890 0.484344
3892 0.437239
3894 0.147107
3895 0.099992
3896 0.552832
3897 0.715783
3899 0.146779
3900 0.400037
3901 0.099992
3902 0.399813
3904 0.184467
3907 0.252865
3912 0.178333
3913 0.62158
3915 0.147102
3919 0.237379
3921 0.078416
3922 1.118455
3923 0.118962
3924 0.099991
3926 0.43102
3927 0.099807
3934 0.431084
3935 0.253196
3936 0.184172
3937 0.131193
3938 0.115592
3940 0.20029
3941 0.300118
3942 0.115879
3943 0.200187
3944 1.184523
3945 0.215588
3947 0.116482
3950 0.351725
3951 0.615359

4771 0.268766
4773 0.384512
4774 0.146936
4775 0.137453
4776 0.299872
4777 0.331295
4778 0.953243
4779 0.163
4780 0.337651
4782 0.400132
4783 0.71493
4784 0.131236
4786 0.653844
4787 0.284457
4788 0.100124
4789 0.078368
4790 0.19994
4791 0.100007
4792 0.23753
4793 0.13157
4794 0.168327
4795 0.11552
4796 0.153324
4797 0.23169
4798 0.100109
4800 0.115872
4802 0.268636
4803 0.280763
4805 1.050186
4806 0.300045
4807 0.831688
4808 0.253166
4809 0.565474
4810 0.119653
4811 0.300407
4813 0.315679
4814 0.131399
4815 0.384958
4816 0.553601
4818 0.347272
4819 0.754105
4820 0.231908
4821 1.385289
4826 0.162563
4827 0.822423
4828 0.131442
4831 0.384357
4832 0.247325
4833 0.234036
4834 0.465661
4836 0.100149
4837 0.100249
4838 0.337325
4840 0.178444
4841 0.122156
4842 0.078294
4843 0.237252
4846 0.115808
4847 0.684379
4848 0.284374
4849 0.131348
4850 0.11555
4851 0.768802
4852 0.084286
4853 1.100889
4854 0.146893
4856 0.269196
4860 0.315591
4861 0.146868
4863 0.217562
4864 0.135738
4871 0.200173
48

5597 0.169176
5598 0.131201
5599 0.152343
5600 1.316129
5601 0.114797
5605 1.047395
5606 0.821458
5607 0.131242
5608 0.184413
5611 0.116091
5612 0.384255
5613 0.247235
5615 0.099947
5617 0.352248
5618 0.18497
5620 0.162297
5621 1.337561
5624 0.199969
5625 0.461867
5626 0.922197
5627 0.200018
5629 0.162468
5630 0.468869
5631 0.263461


In [88]:
tot_tests

[1296,
 138,
 57,
 31,
 103,
 22,
 77,
 25,
 128,
 37,
 99,
 1578,
 53,
 83,
 112,
 42,
 104,
 73,
 51,
 336,
 392,
 158,
 109,
 71,
 371,
 119,
 84,
 73,
 124,
 67,
 1294,
 661,
 171,
 1211,
 86,
 122,
 7313,
 213,
 23,
 72,
 61,
 224,
 51,
 88,
 24,
 153,
 535,
 27,
 37,
 260,
 250,
 32,
 25,
 18,
 60,
 28,
 146,
 69,
 34,
 98,
 211,
 0,
 3514,
 91,
 24,
 28,
 30,
 154,
 175,
 206,
 622,
 48,
 98,
 638,
 159,
 55,
 752,
 61,
 561,
 152,
 23,
 79,
 922,
 97,
 363,
 19,
 69,
 33,
 55,
 1080,
 226,
 199,
 330,
 87,
 99,
 124,
 136,
 36,
 3166,
 32,
 0,
 395,
 154,
 110,
 65,
 180,
 1845,
 103,
 190,
 196,
 761,
 63,
 59,
 2229,
 80,
 148,
 166,
 558,
 152,
 125,
 1403,
 425,
 38,
 24,
 203,
 1017,
 109,
 0,
 86,
 26,
 35,
 33,
 3429,
 176,
 105,
 72,
 26,
 2496,
 188,
 64,
 58,
 56,
 268,
 1505,
 83,
 47,
 50,
 8,
 64,
 46,
 118,
 161,
 1906,
 166,
 295,
 355,
 58,
 362,
 176,
 210,
 87,
 188,
 598,
 182,
 115,
 777,
 118,
 38,
 111,
 70,
 104,
 309,
 3420,
 207,
 20,
 903,
 212,
 422,


In [89]:
pickle.dump(tot_tests,open("total_lab_tests_5050_g.p","wb"))